In [3]:
import numpy as np

int(np.floor(4.4))

4

In [5]:
3 % 4

3

In [8]:
import torch

a = torch.arange(20, dtype=float).reshape(4, 5)

In [12]:
a.T

tensor([[ 0.,  5., 10., 15.],
        [ 1.,  6., 11., 16.],
        [ 2.,  7., 12., 17.],
        [ 3.,  8., 13., 18.],
        [ 4.,  9., 14., 19.]], dtype=torch.float64)

In [11]:
a.T.mean(dim=1)

tensor([ 7.5000,  8.5000,  9.5000, 10.5000, 11.5000], dtype=torch.float64)

In [13]:
torch.matmul(a, a.T.mean(dim=1))

tensor([105.0000, 342.5000, 580.0000, 817.5000], dtype=torch.float64)

In [14]:
a.shape

torch.Size([4, 5])

In [18]:
a.T.mean(dim=1, keepdim=True).shape

torch.Size([5, 1])